In [1]:
import sys
sys.path.append('..')

In [2]:
import pandas as pd
import numpy as np
import pickle
import json
from logics_pack import global_settings, analysis, chemistry, evaluation, frechet_chemnet
import fcd

project_paths = global_settings.build_project_paths(project_dir='../')
expset_obj = global_settings.ExperimentSettings(project_paths['EXPERIMENT_SETTINGS_JSON'])
expset_obj.get_keys()

Using TensorFlow backend.


dict_keys(['kor-pred-best-cv', 'pik3ca-pred-best-cv', 'kor-logics-best-epoch', 'kor-vgpc-best-epoch', 'kor-segler-best-epoch', 'kor-reinv-best-epoch', 'kor-drugex-best-epoch', 'pik3ca-logics-best-epoch', 'pik3ca-segler-best-epoch', 'pik3ca-vgpc-best-epoch', 'pik3ca-reinv-best-epoch', 'pik3ca-drugex-best-epoch'])

In [3]:
## run this cell in case CuDNN library error occurs 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [4]:
fc_ref_model = fcd.load_ref_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2023-04-12 15:30:26.148484: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-04-12 15:30:26.189838: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-12 15:30:26.189871: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: shepherd7
2023-04-12 15:30:26.189878: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: shepherd7
2023-04-12 15:30:26.189970: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 465.19.1
2023-04-12 15:30:26.189998: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 465.19.1
2023-04-12 15:30:26.190004: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 465.19.1
2023-04-12 15:30:26.191234: I tensorflow/core/platform/cpu_featu

In [5]:
SAMPLE_SIZE = 20000  # we sampled and saved 20k for each model
INTDIV_SIZE = 1000  # sample size to be used for intdiv calculation

# we need to fillout the following table
perf_table = pd.DataFrame(index=['validity','uniqueness','novelty','diversity','PredAct','PwSim','FCD','OTD'], 
                        columns=['prior','vgpc','segler','reinv','drugex','logics'])
model_names = perf_table.columns.tolist()
metrics = perf_table.index.tolist()

In [6]:
# recorded epochs for each model
model_e = {
    'prior': 10, 
    'vgpc':int(expset_obj.get_setting('kor-vgpc-best-epoch')),
    'segler':int(expset_obj.get_setting('kor-segler-best-epoch')), 
    'reinv':int(expset_obj.get_setting('kor-reinv-best-epoch')), 
    'drugex':int(expset_obj.get_setting('kor-drugex-best-epoch')),
    'logics':int(expset_obj.get_setting('kor-logics-best-epoch')),
}

In [7]:
# generation file paths
paths_vc = {
    'prior': project_paths['PROJECT_DIR']+'model-prior/prior_vc_e{}.smi'.format(model_e['prior']),
    'vgpc': project_paths['PROJECT_DIR']+'model-kor/vgpc/kor_vgpc_vc_e{}.smi'.format(model_e['vgpc']),
    'segler': project_paths['PROJECT_DIR']+'model-kor/segler/kor_segler_vc_e{}.smi'.format(model_e['segler']),
    'reinv': project_paths['PROJECT_DIR']+'model-kor/reinvent/kor_reinv_vc_e{}.smi'.format(model_e['reinv']),
    'drugex': project_paths['PROJECT_DIR']+'model-kor/drugex/kor_drugex_vc_e{}.smi'.format(model_e['drugex']),
    'logics': project_paths['PROJECT_DIR']+'model-kor/logics/kor_logics_vc_e{}.smi'.format(model_e['logics'])
}
paths_npfps = {
    'prior': project_paths['PROJECT_DIR']+'model-prior/prior_npfps_e{}.npy'.format(model_e['prior']),
    'vgpc': project_paths['PROJECT_DIR']+'model-kor/vgpc/kor_vgpc_npfps_e{}.npy'.format(model_e['vgpc']),
    'segler': project_paths['PROJECT_DIR']+'model-kor/segler/kor_segler_npfps_e{}.npy'.format(model_e['segler']),
    'reinv': project_paths['PROJECT_DIR']+'model-kor/reinvent/kor_reinv_npfps_e{}.npy'.format(model_e['reinv']),
    'drugex': project_paths['PROJECT_DIR']+'model-kor/drugex/kor_drugex_npfps_e{}.npy'.format(model_e['drugex']),
    'logics': project_paths['PROJECT_DIR']+'model-kor/logics/kor_logics_npfps_e{}.npy'.format(model_e['logics'])
}
paths_fc_vecs = {
    'prior': project_paths['PROJECT_DIR']+'model-prior/prior_fcvec_e{}.npy'.format(model_e['prior']),
    'vgpc': project_paths['PROJECT_DIR']+'model-kor/vgpc/kor_vgpc_fcvec_e{}.npy'.format(model_e['vgpc']),
    'segler': project_paths['PROJECT_DIR']+'model-kor/segler/kor_segler_fcvec_e{}.npy'.format(model_e['segler']),
    'reinv': project_paths['PROJECT_DIR']+'model-kor/reinvent/kor_reinv_fcvec_e{}.npy'.format(model_e['reinv']),
    'drugex': project_paths['PROJECT_DIR']+'model-kor/drugex/kor_drugex_fcvec_e{}.npy'.format(model_e['drugex']),
    'logics': project_paths['PROJECT_DIR']+'model-kor/logics/kor_logics_fcvec_e{}.npy'.format(model_e['logics'])
}

In [8]:
# pre-training dataset loading
with open(project_paths['PRETRAINING_DATA_PATH'], 'r') as f:
    pret_smis = [line.strip() for line in f.readlines()]
len(pret_smis)

1583442

In [9]:
# loading predictor 
pred_path = project_paths['PROJECT_DIR'] + "model-kor/predictor/kor_rfr_cv{}.pkl".format(expset_obj.get_setting("kor-pred-best-cv"))
with open(pred_path, 'rb') as f:
    predictor = pickle.load(f)
predictor

RandomForestRegressor()

In [10]:
# loading test set actives (tsa)
affinity_data = pd.read_csv(project_paths['KOR_DATA_PATH'])
with open(project_paths['KOR_FOLD_JSON'], 'r') as f:
    folds = json.load(f)
test_ids = folds[str(global_settings.TEST_FOLD_IDX)]
test_data = affinity_data.iloc[test_ids]

tsa_data = test_data[test_data['affinity']>global_settings.KOR_ACT_THRS]  # active among test set
tsa_smis = tsa_data['smiles'].tolist()
tsa_rdkfps = chemistry.get_fps_from_smilist(tsa_smis)
tsa_fc_vecs = fcd.get_predictions(fc_ref_model, tsa_smis)

In [11]:
# evaluation config objects
evcons = {}
for mn in model_names:
    with open(paths_vc[mn], 'r') as f:
        vc_smis = [line.strip() for line in f.readlines()]
    npfps = np.load(paths_npfps[mn])
    fc_vecs = np.load(paths_fc_vecs[mn])
    evc = evaluation.EvalConfig(
            ssize=SAMPLE_SIZE, vc_smis=vc_smis, npfps=npfps, simmat_size=INTDIV_SIZE, fc_vecs=fc_vecs,
            data_smis=tsa_data, data_rdkfps=tsa_rdkfps, data_fc_vecs=tsa_fc_vecs, ot_repeats=global_settings.OT_CALC_REPEATS
    )
    evcons[mn] = evc

In [12]:
# evaluate and fillout the performance table
for mn in model_names:
    print(mn)
    va, uni, nov, div = evaluation.eval_standard(evcons[mn], pret_smis)
    predact, pwsim, fcdval, otdval = evaluation.eval_optimization(evcons[mn], predictor)
    perf_table[mn]['validity'] = va
    perf_table[mn]['uniqueness'] = uni
    perf_table[mn]['novelty'] = nov
    perf_table[mn]['diversity'] = div
    perf_table[mn]['PredAct'] = predact
    perf_table[mn]['PwSim'] = pwsim
    perf_table[mn]['FCD'] = fcdval
    perf_table[mn]['OTD'] = otdval

prior
vgpc
segler
reinv
drugex
logics


In [13]:
perf_table

,prior,vgpc,segler,reinv,drugex,logics
validity,0.95365,0.8784,0.9685,0.9889,0.9794,0.98485
uniqueness,0.999266,0.995048,0.943108,0.910052,0.996477,0.992689
novelty,0.948318,0.998227,0.997208,0.941886,0.991598,0.986447
diversity,0.88942,0.801062,0.853575,0.870009,0.83221,0.85734
PredAct,5.952136,7.042889,8.310055,7.02482,7.111535,7.571087
PwSim,0.109935,0.106264,0.129759,0.123477,0.124162,0.131003
FCD,27.326926,38.828221,22.366105,25.968452,30.598097,22.205187
OTD,5.384638,5.852848,5.100843,5.102824,5.383503,4.952175
